## Mimicking Bertrand Russell's writing style with LSTM

In [1]:
import sys
import pandas as pd
import numpy as np
from scipy import stats
import researchpy as rp
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob
import os
import math
# import warnings
# warnings.filterwarnings('ignore')
os.chdir('../Book files/books')


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import string
import collections
import tensorflow as tf
# from tensorflow import keras
import keras
import re
import copy

Using TensorFlow backend.


## 1. Generative Models for Text

#### (C) LSTM: Train an LSTM to mimic Russell’s style and thoughts:
##### i. Concatenate your text files to create a corpus of Russell’s writings.


In [ ]:
# TPP:The Problems of Philosophy
# TAM: The Analysis of Mind
# MLOE:  Mysticism and Logic and Other Essays
# OKEWFSMP: Our Knowledge of the External World as a Field for Scientific Method in Philosophy

In [3]:
dataFiles = ["TPP.txt","TAM.txt","MLOE.txt","OKEWFSMP.txt" ]            

resultFile = open('corpus.txt', 'w')
for fileItem in dataFiles:
        with open(fileItem, 'r', errors='ignore') as ff:
            for line in ff:
                resultFile.write(line)

resultFile.close()

corpus = open('corpus.txt', 'r').read()


In [148]:
print(corpus [:1000])
print(len(corpus))

ï»¿CHAPTER I. APPEARANCE AND REALITY

Is there any knowledge in the world which is so certain that no
reasonable man could doubt it? This question, which at first sight might
not seem difficult, is really one of the most difficult that can
be asked. When we have realized the obstacles in the way of a
straightforward and confident answer, we shall be well launched on the
study of philosophy--for philosophy is merely the attempt to answer
such ultimate questions, not carelessly and dogmatically, as we do in
ordinary life and even in the sciences, but critically, after exploring
all that makes such questions puzzling, and after realizing all the
vagueness and confusion that underlie our ordinary ideas.

In daily life, we assume as certain many things which, on a closer
scrutiny, are found to be so full of apparent contradictions that only a
great amount of thought enables us to know what it is that we really may
believe. In the search for certainty, it is natural to begin with our
present

##### ii. Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0, 1], because LSTM


In [20]:
# import nltk
from nltk.tokenize import RegexpTokenizer
text = corpus.encode("utf8").decode("ascii",'ignore')


# lowercasing all the text + removing all the punctuations
text = corpus.lower()
# words = nltk.word_tokenize(text)
tokenizer = RegexpTokenizer(r'\w+')
text = tokenizer.tokenize(text)

# print(len(text))
text =' '.join(text)
print(text [:1000])


ï chapter i appearance and reality is there any knowledge in the world which is so certain that no reasonable man could doubt it this question which at first sight might not seem difficult is really one of the most difficult that can be asked when we have realized the obstacles in the way of a straightforward and confident answer we shall be well launched on the study of philosophy for philosophy is merely the attempt to answer such ultimate questions not carelessly and dogmatically as we do in ordinary life and even in the sciences but critically after exploring all that makes such questions puzzling and after realizing all the vagueness and confusion that underlie our ordinary ideas in daily life we assume as certain many things which on a closer scrutiny are found to be so full of apparent contradictions that only a great amount of thought enables us to know what it is that we really may believe in the search for certainty it is natural to begin with our present experiences and in s

In [31]:
# initializing bad_chars_list
bad_chars = ['µ', '¼','å', '_','²','½', '³','á', 'å', 'º', '¹', 'ƒ', 'µ', 'î','â', 
             'ã', 'ï', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '\n', '_', 'â', '\xa0','ã', 'î']
for i in bad_chars :
    text = text.replace(i, '')

In [32]:
count = collections.Counter(text)
# value = print.pformat(count)
print(count)

Counter({' ': 273218, 'e': 160450, 't': 123709, 'i': 102032, 'a': 95505, 'o': 93371, 's': 91431, 'n': 89120, 'h': 68627, 'r': 66178, 'l': 47580, 'c': 44913, 'd': 37002, 'u': 34004, 'f': 31151, 'm': 30673, 'p': 26752, 'y': 21720, 'z': 388})


In [33]:
a= list(count.items())
# sorted(a, reverse=True)
a.sort(key = lambda x: x[1]) 
a

[('z', 388),
 ('y', 21720),
 ('p', 26752),
 ('m', 30673),
 ('f', 31151),
 ('u', 34004),
 ('d', 37002),
 ('c', 44913),
 ('l', 47580),
 ('r', 66178),
 ('h', 68627),
 ('n', 89120),
 ('s', 91431),
 ('o', 93371),
 ('a', 95505),
 ('i', 102032),
 ('t', 123709),
 ('e', 160450),
 (' ', 273218)]

In [34]:
label_char = {}
for index, char in enumerate(a):
    label_char[char[0]] = index
# label_char

{'z': 0,
 'y': 1,
 'p': 2,
 'm': 3,
 'f': 4,
 'u': 5,
 'd': 6,
 'c': 7,
 'l': 8,
 'r': 9,
 'h': 10,
 'n': 11,
 's': 12,
 'o': 13,
 'a': 14,
 'i': 15,
 't': 16,
 'e': 17,
 ' ': 18}

In [55]:
int_char = {}
for index, char in enumerate(a):
    int_char[index] = char[0]
# int_char

{0: 'z',
 1: 'y',
 2: 'p',
 3: 'm',
 4: 'f',
 5: 'u',
 6: 'd',
 7: 'c',
 8: 'l',
 9: 'r',
 10: 'h',
 11: 'n',
 12: 's',
 13: 'o',
 14: 'a',
 15: 'i',
 16: 't',
 17: 'e',
 18: ' '}

In [35]:
ASCII = {}
for index, char in enumerate(a):
    ASCII[char[0]] = ord(char[0])
# ASCII   
# len(ASCII)

{'z': 122,
 'y': 121,
 'p': 112,
 'm': 109,
 'f': 102,
 'u': 117,
 'd': 100,
 'c': 99,
 'l': 108,
 'r': 114,
 'h': 104,
 'n': 110,
 's': 115,
 'o': 111,
 'a': 97,
 'i': 105,
 't': 116,
 'e': 101,
 ' ': 32}

In [37]:
scaleASCII = {}
scaleval= MinMaxScaler().fit_transform(np.array(list(ASCII.values())).reshape(-1, 1))
for index in range(len(scaleval)):
    scaleASCII[list(ASCII.keys())[index][0]] = scaleval[index][0]

# scaleASCII

##### iii. Choose a window size, e.g., W = 100.
##### iv. 
Inputs to the network will be the first W −1 = 99 characters of each sequence,
and the output of the network will be the Wth character of the sequence.
Basically, we are training the network to predict each character using the 99
characters that precede it. Slide the window in strides of S = 1 on the text.
For example, if W = 5 and S = 1 and we want to train the network with the
sequence ABRACADABRA, The first input to the network will be ABRA
and the corresponding output will be C. The second input will be BRAC and
the second output will be A, etc

In [38]:
Wsize = 99  # window size -1 

X_data= []
y_data= []
for i in range(0,len(text)- Wsize):
    temp =text[i:i + Wsize]
    lasttemp = text[i+ Wsize]
    X_data.append([scaleASCII[char]for char in temp])
    y_data.append(label_char[lasttemp])
#     y_data.append(ASCII[lasttemp])

In [29]:
X = np.reshape(X_data, (len(X_data), Wsize, 1)) #number of patterns
y = keras.utils.to_categorical(y_data)
print(X.shape, y.shape)


(1437725, 99, 1) (1437725, 19)


##### v. Note that the output has to be encoded using a one-hot encoding scheme with
N = 256 (or less) elements. This means that the network reads integers, but
outputs a vector of N = 256 (or less) elements.

##### vi.Use a single hidden layer for the LSTM with N = 256 (or less) memory units.
##### vii. Use a Softmax output layer to yield a probability prediction for each of the characters between 0 and 1. 
This is actually a character classification problem
with N classes. Choose log loss (cross entropy) as the objective function for
the network (research what it means).

##### viii. We do not use a test dataset.
We are using the whole training dataset to learn the probability of each character in a sequence. We are not seeking for
a very accurate model. Instead we are interested in a generalization of the dataset that can mimic the gist of the text.
###### ix. Choose a reasonable number of epochs for training, considering your computational power (e.g., 30, although the network will need more epochs to yield a better model).
##### x. Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss

In [84]:
#creating the model

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))
# model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))

model.compile( optimizer='adam', loss='categorical_crossentropy')
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dense (Dense)                (None, 19)                4883      
Total params: 269,075
Trainable params: 269,075
Non-trainable params: 0
_________________________________________________________________


In [85]:
# checkpoint
# filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

filepath="weights-improvement1-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=15, batch_size=128, callbacks=callbacks_list)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/15
    896/1437725 [..............................] - ETA: 2:29:20 - loss: 2.8448

KeyboardInterrupt: 

xi. Use the network with the best weights to generate 1000 characters, using the
following text as initialization of the network:
There are those who take mental phenomena naively, just as they
would physical phenomena. This school of psychologists tends not to
emphasize the object.

In [83]:
#generating text via a seed text

init_seed_sent = 'there are those who take mental phenomena naively, just as they would physical phenomena. this school of psychologists tends not to emphasize the object.' 
print(len(init_seed_sent))

153


In [82]:
#clean init_text
init_seed_sent = init_seed_sent.encode("utf8").decode("ascii",'ignore')

seed = init_seed_sent.lower()
seed = tokenizer.tokenize(seed)
seed_init =' '.join(seed)
print(len(seed_init))
print(seed_init)


gen = copy.copy(seed_init)


150
there are those who take mental phenomena naively just as they would physical phenomena this school of psychologists tends not to emphasize the object


In [78]:
pattern = [scaleASCII[char] for char in gen][-99:]
for i in range(1000):
    XX = np.reshape(pattern, (1, len(pattern), 1))
    pred = model.predict(XX, verbose=0)
    charIndex = np.argmax(pred)
    char = int_char[np.argmax(pred)]
    gen += char
    pattern.append(scaleASCII[char])
    pattern = pattern[1:len(pattern)]
    
print(gen)

150


'there are those who take mental phenomena naively just as they would physical phenomena this school of psychologists tends not to emphasize the object'

#### citation:
- https://stackoverflow.com/questions/17749058/combine-multiple-text-files-into-one-text-file-using-python
- http://www.nltk.org/api/nltk.tokenize.html
- https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
- https://machinelearningmastery.com/check-point-deep-learning-models-keras/
- https://en.wikibooks.org/wiki/Python_Programming/Text#:~:text=To%20get%20the%20ASCII%20code,use%20the%20ord()%20function.&text=To%20get%20the%20character%20encoded,use%20the%20chr()%20function.&text=To%20know%20if%20all%20the,use%20the%20isalnum()%20function.
- https://www.geeksforgeeks.org/python-removing-unwanted-characters-from-string/
-https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
- https://stackoverflow.com/questions/57081411/i-want-to-create-a-corpus-in-python-from-multiple-text-files